## Instructions:

This notebook contains our final model. You can replace the dataset name with your owns but the same category. Then run the file from start to end. Thanks!

In [1]:
import pandas as pd
import numpy as np
import re
import spacy
from scipy.spatial.distance import cosine
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score,KFold, StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from keras.preprocessing.text import Tokenizer
from typing import List
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers.recurrent import SimpleRNN
from keras.layers import Flatten, Masking

from numpy import array, argmax, asarray, zeros

import keras
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers import Flatten, Masking, Embedding
from keras.layers import Activation, Dense
from tensorflow.python.framework.ops import disable_eager_execution
from keras.models import Sequential                                       
from keras.utils.vis_utils import plot_model
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.models import Model

import warnings
warnings.filterwarnings('ignore')

# Load Dataset

## Product

In [2]:
%%time
# load product information
product = pd.read_excel('Behold+product+data+04262021.xlsx')
product.head(3)

CPU times: user 13.3 s, sys: 249 ms, total: 13.6 s
Wall time: 13.7 s


,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,NaN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,NaN,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,Mid-length dress with ruffles and adjustable s...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,True
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,NaN,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False


In [3]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61355 entries, 0 to 61354
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           61355 non-null  object
 1   brand                61355 non-null  object
 2   brand_category       60896 non-null  object
 3   name                 61354 non-null  object
 4   details              9200 non-null   object
 5   created_at           61355 non-null  object
 6   brand_canonical_url  61355 non-null  object
 7   description          51238 non-null  object
 8   brand_description    51234 non-null  object
 9   brand_name           61354 non-null  object
 10  product_active       61355 non-null  bool  
dtypes: bool(1), object(10)
memory usage: 4.7+ MB


## Brand

In [4]:
# load brand information
brand = pd.read_csv('behold_brands USC.csv')
brand.head(3)

,brand_id,brand,brand_value,bio,quote,quote_attribute,intro,lifestyle_copy,short_bio,listing_bio
0,01ESKR0CH2KYC7KBNTN0S38EQA,Mari Giudicelli,Handmade / Artisan Crafted,NaN,NaN,NaN,NaN,NaN,NaN,Behold Mari Giudicelli! This Brazilian shoe de...
1,01ESKR0CH2KYC7KBNTN0S38EQA,Mari Giudicelli,Sustainable,NaN,NaN,NaN,NaN,NaN,NaN,Behold Mari Giudicelli! This Brazilian shoe de...
2,01ESKR0CH2KYC7KBNTN0S38EQA,Mari Giudicelli,Women Owned,NaN,NaN,NaN,NaN,NaN,NaN,Behold Mari Giudicelli! This Brazilian shoe de...


In [5]:
brand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   brand_id         162 non-null    object
 1   brand            162 non-null    object
 2   brand_value      154 non-null    object
 3   bio              157 non-null    object
 4   quote            157 non-null    object
 5   quote_attribute  154 non-null    object
 6   intro            154 non-null    object
 7   lifestyle_copy   156 non-null    object
 8   short_bio        157 non-null    object
 9   listing_bio      157 non-null    object
dtypes: object(10)
memory usage: 12.8+ KB


## Outfit

In [6]:
outfit = pd.read_csv('outfit_combinations USC.csv')
outfit.head(5)

,outfit_id,product_id,outfit_item_type,brand,product_full_name
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2ZFDYRYY5TRQZJTBD,shoe,Tory Burch,Penelope Mid Cap Toe Pump
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt


In [7]:
outfit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   outfit_id          5291 non-null   object
 1   product_id         5291 non-null   object
 2   outfit_item_type   5291 non-null   object
 3   brand              5291 non-null   object
 4   product_full_name  5291 non-null   object
dtypes: object(5)
memory usage: 206.8+ KB


## Expert Tags

In [8]:
tags = pd.read_csv('usc_additional_tags USC.csv')
tags.head(3)

,product_id,product_color_id,attribute_name,attribute_value
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,linenblend
1,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,cottonblend
2,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,modern


In [9]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97420 entries, 0 to 97419
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        97420 non-null  object
 1   product_color_id  97420 non-null  object
 2   attribute_name    97420 non-null  object
 3   attribute_value   97420 non-null  object
dtypes: object(4)
memory usage: 3.0+ MB


# EDA

There is no duplicate product id in product.csv

In [10]:
len(product.product_id.unique())==len(product)

True

We have more brands in product.csv compared to brand.csv and outfit.csv

In [11]:
# number of unique brands in product.csv
len(product.brand.unique())

386

In [12]:
# number of unique brands in brand.csv
len(brand.brand.unique())

74

In [13]:
# number of unique brands in outfit.csv
len(outfit.brand.unique())

306

There are 6 different brand values which could be set as a new feature later on

In [14]:
brand.brand_value.value_counts()

Women Owned                   47
Emerging                      41
Handmade / Artisan Crafted    29
Sustainable                   27
BIPOC Owned                    6
Vegan                          4
Name: brand_value, dtype: int64

There are 7 different outfit item types, and five different outfit types:
- shoe
- accessory
- top
- bottom
- onepiece

In [15]:
outfit.outfit_item_type.value_counts()

shoe          1149
accessory1    1064
accessory2     978
top            950
bottom         928
onepiece       221
accessory3       1
Name: outfit_item_type, dtype: int64

There are 86 different attribute types, each type is assocaites with several values. Attributes could be added to features for different products later on.

In [16]:
len(tags.attribute_name.unique())

86

In [17]:
tags.attribute_name.value_counts()[:20]

style               14462
occasion            12624
materialclothing     5589
category             5522
primarycolor         5466
gender               5433
fit                  3814
sizing               3701
drycleanonly         3488
sleevelength         2619
additionalcolor      2572
neckline             2417
lengthtop            1837
pattern              1604
subcategorytop       1387
uppermaterial        1252
subcategoryshoe      1246
shoewidth            1221
toestyle             1217
closureshoe          1163
Name: attribute_name, dtype: int64

# Part 1: classification

**Problem Statement:**
>A common problem for Behold is the extremely manual classification of new products 
    into their specific brands. Behold has supplier relationships from many global clothing 
    vendors and marketplaces, but has found that the bottleneck for scaling out the number 
    of products they can sell is identifying the brands associated with each new product. Use 
    the brands’ biographical description (behold_brands.csv), product descriptions
    (products.csv) and any externally collected metadata, **build an NLP classification
    model to predict which brand a new product should be assigned**

**Solution Pipeline:**

1. Preprocess the text (e.g., lemmatization, grouping, tokenize)
2. Feature engineering (e.g., using tags, product attributes)
    - Feature creation
3. Vectorization (either via count vectorization or via embeddings)
4. train/test split and modeling (try different models and compare the performance)

## Preprocessing

In [18]:
%%time
## need to download 'en_core_web_lg' first
nlp = spacy.load('en_core_web_lg',disable=['ner','parser'])

CPU times: user 1.44 s, sys: 812 ms, total: 2.25 s
Wall time: 2.87 s


### lemmatization

In [19]:
def spacy_lem(data, field):
    '''
    do lemmatization for data[field]
    '''
    for i in range(len(data)):
        if type(data.loc[i,field])==str:
            rows = []
            doc = nlp(data.loc[i,field])
            for token in doc:
                rows.append(token.lemma_)
            data.loc[i,field] = ' '.join(rows)

In [20]:
%%time
for field in ['description','details','name','brand_category']:
    spacy_lem(product,field)

CPU times: user 15min 4s, sys: 23.2 s, total: 15min 27s
Wall time: 15min 30s


In [21]:
%%time
for field in brand.columns[2:]:
    spacy_lem(brand,field)

CPU times: user 4.7 s, sys: 134 ms, total: 4.83 s
Wall time: 4.86 s


In [22]:
product_cp = product.copy()
brand_cp = brand.copy()

### Regex Cleaning

In [23]:
def removePunctuation(text, punctuations=string.punctuation+"``"+"’"+"”"):
    words=nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in punctuations]
    cleanedText = " ".join(newWords)
    return cleanedText

In [24]:
nltk_stopwords = set(stopwords.words("English"))
def removeStopwords(text, stopwords=nltk_stopwords):
    words = nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in stopwords]
    cleanedText = " ".join(newWords)
    return cleanedText

In [25]:
%%time
for col in ['description','details']:
    product[col] = product[col].str.replace('\s\s',' ')
    product[col] = product[col].fillna('UNKNOWN_TOKEN')
    product[col] = product[col].apply(removePunctuation)
    product[col] = product[col].apply(removeStopwords)

for col in brand.columns[2:]:
    brand[col] = brand[col].str.replace('\s\s',' ')
    brand[col] = brand[col].fillna('UNKNOWN_TOKEN')
    brand[col] = brand[col].apply(removePunctuation)
    brand[col] = brand[col].apply(removeStopwords)

CPU times: user 41.6 s, sys: 111 ms, total: 41.7 s
Wall time: 41.8 s


## Feature Engineering

### Concatenating data

In [26]:
# put different brand values into one columns
df = brand.groupby('brand').agg({'brand_value':list}).reset_index().rename(columns = {'brand_value':'value_brand'})
df_brand = df.merge(brand,on='brand',how='left')[['brand', 'value_brand', 'bio', 'quote',
       'quote_attribute', 'intro', 'lifestyle_copy', 'short_bio',
       'listing_bio']]
df_brand.sample(3)

,brand,value_brand,bio,quote,quote_attribute,intro,lifestyle_copy,short_bio,listing_bio
107,Merlette,"[sustainable, Handmade Artisan Crafted, woman,...",honor responsibility come bring new design wor...,thing pack know wear,New York Times,Merlette Brooklyn base atelier create thoughtf...,Marina Cortbawi create piece woman keep come b...,Merlette New York,behold Merlette Brooklyn base atelier found Ma...
43,Chufy,"[woman, sustainable, emerge]",launch 2017 Chufy feature ready wear infuse hi...,real voyage discovery consist seek new landsca...,Marcel Proust,season Chufy focus new destination culture col...,sunset beach flow dress ...,discover Chufy 's travel inspire collection,behold Chufy found well season traveler style ...
51,E.L.V. Denim,"[sustainable, emerge, woman, Handmade Artisan ...",work local atelier East London Anna Foster ens...,sustainability ultimate part marry aesthetic w...,Anna Foster Founder Creative Director,E.L.V DENIM take unwanted jean destine landfil...,Boyfriend Jean staple wardrobe style unique je...,zero waste upcycled denim brand design produce...,behold E.L.V Denim stand East London Vintage b...


In [27]:
# combine product and brand
product_brand = product.merge(df_brand,on='brand',how='left')
product_brand = product_brand.drop_duplicates(subset=['product_id'], keep='first', inplace=False).\
                reset_index().drop(columns=['index'])
product_brand.head(2)

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active,value_brand,bio,quote,quote_attribute,intro,lifestyle_copy,short_bio,listing_bio
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,unknown,Khadi Stripe Shirt - our signature shirt,UNKNOWN_TOKEN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,signature khadi shirt available black white ea...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True,"[sustainable, Handmade Artisan Crafted, woman]",two modern minimalist take caftan tunic easy d...,two emphasize importance sustainable craftsman...,Monica Patel Cohn Designer founder,chic modern piece gorgeous handwoven fabric,think two feel certain energy bliss calm warmt...,art Sari,behold two founder Monica Patel Cohn launch fr...
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,unknown,ruffle market dress LOOPY PINK SISTINE TOMATO,UNKNOWN_TOKEN,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,mid length dress ruffle adjustable strap bias ...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,True,"[woman, emerge]",well ready wear collection Collina Strada plat...,season season goal remain encourage self refle...,Hillary Taymour,Collina Strada platform social issue awareness...,Collina Strada embody humor youth brand dna fi...,easy wear clothe imbue fearlessly fluid attitude,behold Hillary Taymor master tie dye ethical w...


In [28]:
product_brand.shape

(61355, 19)

In [29]:
tags['attribute_pair']=tags['attribute_name']+": "+tags['attribute_value']
tags.head(1)

,product_id,product_color_id,attribute_name,attribute_value,attribute_pair
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,linenblend,materialclothing: linenblend


In [30]:
tag_combined = tags.groupby('product_id').agg({'attribute_pair':list}).reset_index()
tag_combined.sample(3)

,product_id,attribute_pair
3423,01E4VDXNV313XWGVQ8GCEWKN1W,"[primarycolor: beiges, style: modern, material..."
982,01E2FDJ3CQ5A8F7H6DSTPYHW07,"[primarycolor: blacks, materialclothing: alpac..."
1580,01E2KY7HRN6JPXV45NHF5FV9G7,"[primarycolor: blacks, materialclothing: woolb..."


In [31]:
product_tag_brand = product_brand.merge(tag_combined,on='product_id',how='left')
product_tag_brand = product_tag_brand.drop(columns=['created_at',
                                                    'brand_canonical_url','brand_description',
                                                    'brand_name','product_active'
                                                   ])
product_tag_brand.isna().sum()

product_id             0
brand                  0
brand_category       459
name                   1
details                0
description            0
value_brand        18680
bio                18680
quote              18680
quote_attribute    18680
intro              18680
lifestyle_copy     18680
short_bio          18680
listing_bio        18680
attribute_pair     57575
dtype: int64

In [32]:
product_tag_brand.shape

(61355, 15)

In [33]:
product_tag_brand = product_tag_brand.fillna('UNKNOWN_TOKEN')

### Feature Creation

In [34]:
product_tag_brand.head(3)

,product_id,brand,brand_category,name,details,description,value_brand,bio,quote,quote_attribute,intro,lifestyle_copy,short_bio,listing_bio,attribute_pair
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,unknown,Khadi Stripe Shirt - our signature shirt,UNKNOWN_TOKEN,signature khadi shirt available black white ea...,"[sustainable, Handmade Artisan Crafted, woman]",two modern minimalist take caftan tunic easy d...,two emphasize importance sustainable craftsman...,Monica Patel Cohn Designer founder,chic modern piece gorgeous handwoven fabric,think two feel certain energy bliss calm warmt...,art Sari,behold two founder Monica Patel Cohn launch fr...,UNKNOWN_TOKEN
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,unknown,ruffle market dress LOOPY PINK SISTINE TOMATO,UNKNOWN_TOKEN,mid length dress ruffle adjustable strap bias ...,"[woman, emerge]",well ready wear collection Collina Strada plat...,season season goal remain encourage self refle...,Hillary Taymour,Collina Strada platform social issue awareness...,Collina Strada embody humor youth brand dna fi...,easy wear clothe imbue fearlessly fluid attitude,behold Hillary Taymor master tie dye ethical w...,UNKNOWN_TOKEN
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,unknown,ibi slip on Raw Red Knit Sneaker woman,UNKNOWN_TOKEN,ibi slip Raw Red Knit Sneaker woman,"[BIPOC, emerge, Handmade Artisan Crafted, sust...",embrace personal style status quo trend empowe...,CARIUMA end search truly sustainable ultra com...,forbe,believe style comfort sustainability synonymou...,CARIUMA sustainability style comfort synonymou...,believe make thing way well people planet,behold Cariuma brazilian brand found good frie...,UNKNOWN_TOKEN


In [35]:
# manufacturing location features
pd.set_option('display.max_rows', 30)
location = product_tag_brand['description'].str.\
       extract(r'\b(?:made|make)\s?(?:in)?\s?(?!Lark|Organic|Cardigan|Charli|ATM|Holiday|Fall|Far)([A-Z]\w+\s?(?!Dry|Emeline|Michelle|Stretch|Models?|Style|Fit|Cotton|Swim|Crown|Madeleine|Phoebe|Limited|Army|Ally|Width|Knitted|Embossed|Regular|Ella|Micro)(?:[A-Z][a-z]\w+)?)\b')
product_tag_brand['manuf_location'] = location[0]

In [36]:
# change from list to str
product_tag_brand['value_brand'] = [" ".join(i) if i!='UNKNOWN_TOKEN' else 'UNKNOWN_TOKEN' for i in product_tag_brand['value_brand']]

In [37]:
# brand value features （!!could only be used for part 2 problem）
product_tag_brand['is_handmade'] = ~pd.isna(product_tag_brand['value_brand'].str.extract(r'\b(Handmade)\b'))
product_tag_brand['is_artisan_crafted'] = ~pd.isna(product_tag_brand['value_brand'].str.extract(r'\b(Artisan Crafted)\b'))
product_tag_brand['is_sustainable'] = ~pd.isna(product_tag_brand['value_brand'].str.extract(r'\b(sustainable)\b'))
product_tag_brand['is_emerging'] = ~pd.isna(product_tag_brand['value_brand'].str.extract(r'\b(emerge)\b'))
product_tag_brand['is_vegan'] = ~pd.isna(product_tag_brand['value_brand'].str.extract(r'\b(Vegan)\b'))
product_tag_brand['is_BIPOC'] = ~pd.isna(product_tag_brand['value_brand'].str.extract(r'\b(BIPOC)\b'))
product_tag_brand['is_woman_owned'] = ~pd.isna(product_tag_brand['value_brand'].str.extract(r'\b(woman)\b'))

In [38]:
# change from list to str
product_tag_brand['attribute_pair'] = [" ".join(i) if i!='UNKNOWN_TOKEN' else 'UNKNOWN_TOKEN' for i in product_tag_brand['attribute_pair']]

In [39]:
# occassion features
product_tag_brand['is_beach'] = ~pd.isna(product_tag_brand['description'].str.extract(r'\b(beach)\b')) + (~pd.isna(product_tag_brand['details'].str.extract(r'\b(beach)\b')))
product_tag_brand['is_city'] = pd.notna(product_tag_brand['description'].str.extract(r'\b(city)\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b(city)\b'))
product_tag_brand['is_holiday'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:vacation|holiday|getaway|weekend))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:vacation|holiday|getaway|weekend))\b'))
product_tag_brand['is_romance'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:romantic|romance))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:romantic|romance))\b'))
product_tag_brand['is_chic'] = pd.notna(product_tag_brand['description'].str.extract(r'\b(chic)\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b(chic)\b'))
product_tag_brand['is_comfortable'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:comfort|cozy))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:comfort|cozy))\b'))
product_tag_brand['is_weekday'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:weekday))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:weekday))\b'))
product_tag_brand['is_asymmetrical'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:asymmetrical))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:asymmetrical))\b'))
product_tag_brand['is_timeless'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:timeless))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:timeless))\b'))
product_tag_brand['is_classic'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:classics?))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:classics?))\b'))
product_tag_brand['is_swim'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:swim|swimming))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:swim|swimming))\b'))
product_tag_brand['is_casual'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:casual))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:casual))\b'))
product_tag_brand['is_pajama'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:pajama))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:pajama))\b'))
product_tag_brand['is_sophisticated'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:sophisticated?))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:sophisticated?))\b'))
product_tag_brand['is_modern'] = pd.notna(product_tag_brand['description'].str.extract(r'\b((?:modern|contemporary))\b')) + pd.notna(product_tag_brand['details'].str.extract(r'\b((?:modern|contemporary))\b'))

In [40]:
# material features
pd.set_option('display.max_rows', 30)
material = product_tag_brand['description'].str.extract(r'\b(100\s\w+)\b')
product_tag_brand['material'] = material[0].str.lower()

In [41]:
product_tag_brand.sample(3)

,product_id,brand,brand_category,name,details,description,value_brand,bio,quote,quote_attribute,...,is_weekday,is_asymmetrical,is_timeless,is_classic,is_swim,is_casual,is_pajama,is_sophisticated,is_modern,material
10824,01EF50P99SZR3NKYJCA9A824KK,lemlem,unknown,Eshe Twist Turban,UNKNOWN_TOKEN,complete LEMLEM look ESHETWIST TURBAN feature ...,sustainable BIPOC woman Handmade Artisan Craft...,lemlem woman 's resort wear brand make entirel...,want inject energy fuel back artisan artisanal...,Liya Kebede,...,False,False,False,False,False,False,False,False,False,100 cotton
48869,01ED57W56PCQBEWPYXDP6QT49D,lemlem,unknown,boy Selina Short Sleeve Popover - Blue,UNKNOWN_TOKEN,style details boy Selina Short Sleeve Popover ...,sustainable BIPOC woman Handmade Artisan Craft...,lemlem woman 's resort wear brand make entirel...,want inject energy fuel back artisan artisanal...,Liya Kebede,...,False,False,False,False,False,False,False,False,False,NaN
19577,01F3E1T87SWSMMFMQENWZB9V2Y,Want Les Essentiels,unknown,Pelli Ankle Socks,UNKNOWN_TOKEN,Pelli Ankle Socks equal part essential indulge...,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,...,False,False,False,True,False,False,False,False,False,NaN


### Grouping

In [42]:
# 'manuf_location'
product_tag_brand.manuf_location = product_tag_brand.manuf_location.str.replace(r'\bLA\b','Los Angeles')
product_tag_brand.manuf_location = product_tag_brand.manuf_location.str.replace(r'\bNY｜NYC\b','New York')
product_tag_brand.manuf_location = product_tag_brand.manuf_location.str.replace(r'\bUS\b','USA')
product_tag_brand.manuf_location = product_tag_brand.manuf_location.str.replace(r'\bItalian|ItalySmall\b','Italy')
product_tag_brand.manuf_location = product_tag_brand.manuf_location.str.replace(r'\bCN\b','China')

In [43]:
df=product_tag_brand.copy()

In [44]:
df.fillna('UNKNOWN_TOKEN',inplace=True)
df.brand = df.brand.astype(str)

In [45]:
df.brand_category = df.brand_category.str.replace('Unknown','UNKNOWN_TOKEN')
df.head()

,product_id,brand,brand_category,name,details,description,value_brand,bio,quote,quote_attribute,...,is_weekday,is_asymmetrical,is_timeless,is_classic,is_swim,is_casual,is_pajama,is_sophisticated,is_modern,material
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,unknown,Khadi Stripe Shirt - our signature shirt,UNKNOWN_TOKEN,signature khadi shirt available black white ea...,sustainable Handmade Artisan Crafted woman,two modern minimalist take caftan tunic easy d...,two emphasize importance sustainable craftsman...,Monica Patel Cohn Designer founder,...,False,False,False,False,False,False,False,False,False,UNKNOWN_TOKEN
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,unknown,ruffle market dress LOOPY PINK SISTINE TOMATO,UNKNOWN_TOKEN,mid length dress ruffle adjustable strap bias ...,woman emerge,well ready wear collection Collina Strada plat...,season season goal remain encourage self refle...,Hillary Taymour,...,False,False,False,False,False,False,False,False,False,100 rose
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,unknown,ibi slip on Raw Red Knit Sneaker woman,UNKNOWN_TOKEN,ibi slip Raw Red Knit Sneaker woman,BIPOC emerge Handmade Artisan Crafted sustainable,embrace personal style status quo trend empowe...,CARIUMA end search truly sustainable ultra com...,forbe,...,False,False,False,False,False,False,False,False,False,UNKNOWN_TOKEN
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,unknown,ibi slip on Black Knit Sneaker woman,UNKNOWN_TOKEN,ibi slip Black Knit Sneaker woman,BIPOC emerge Handmade Artisan Crafted sustainable,embrace personal style status quo trend empowe...,CARIUMA end search truly sustainable ultra com...,forbe,...,False,False,False,False,False,False,False,False,False,UNKNOWN_TOKEN
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,UNKNOWN_TOKEN,UNKNOWN_TOKEN,BIPOC emerge Handmade Artisan Crafted sustainable,embrace personal style status quo trend empowe...,CARIUMA end search truly sustainable ultra com...,forbe,...,False,False,False,False,False,False,False,False,False,UNKNOWN_TOKEN


In [46]:
df.to_csv('after_feature_creation.csv',index=None)

In [47]:
df = pd.read_csv('after_feature_creation.csv')

### Onehot encoding

In [48]:
# one-hot encoding
combined = pd.get_dummies(df[['manuf_location', 'is_beach', 'is_city', 'is_holiday',
       'is_romance', 'is_chic', 'is_comfortable', 'is_weekday',
       'is_asymmetrical', 'is_timeless', 'is_classic', 'is_swim', 'is_casual',
       'is_pajama', 'is_sophisticated', 'is_modern', 'material']])

In [49]:
onehot_df = np.concatenate([df[['brand']],combined],axis=1)

In [50]:
combined.shape

(61355, 686)

### Word embedding

In [51]:
# e.g. docs = df['description']
# using glove.txt
def word_embedding(docs):
    '''
    input: docs (e.g., df['description']) needed to do word embedding
    output: corresponding vectors
    '''
    # Tokenize the Text
    tokenizer = Tokenizer(num_words=10000, oov_token="UNKNOWN_TOKEN")
    tokenizer.fit_on_texts(docs)

    # Integer Encode Tokens
    def integer_encode_documents(docs, tokenizer):
        return tokenizer.texts_to_sequences(docs)

    # Get Max Length Per Token
    def get_max_token_length_per_doc(docs: List[List[str]])-> int:
        return max(list(map(lambda x: len(x.split()), docs)))
    ## get the max length in terms of token length
    max_length = get_max_token_length_per_doc(docs)

    MAX_SEQUENCE_LENGTH = 300
    ## integer encode the documents
    encoded_docs = integer_encode_documents(docs, tokenizer)
    ## this is a list of lists, the numbers represent the index position of that word.
    ## for instance, 33 means the 33rd word in the vocabulary
    padded_docs = pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    
    def load_glove_vectors():
        embeddings_index = {}
        with open('glove.6B.100d.txt') as f:
            for line in f:
                values = line.split()
                word = values[0]
                coefs = asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
        print('Loaded %s word vectors.' % len(embeddings_index))
        return embeddings_index
    embeddings_index = load_glove_vectors()
    
    VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)
    
    # create a weight matrix for words in training docs
    embedding_matrix = zeros((VOCAB_SIZE, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: # check that it is an actual word that we have embeddings for
            embedding_matrix[i] = embedding_vector    

    
    return tokenizer, padded_docs, embedding_matrix, MAX_SEQUENCE_LENGTH

In [52]:
# don't use pre-trained embeddings
def word_embedding_2(docs):
    '''
    input: docs (e.g., df['description','details']) needed to do word embedding
    output: corresponding vectors
    '''
    # Tokenize the Text
    tokenizer = Tokenizer(num_words=10000, oov_token="UNKNOWN_TOKEN")
    tokenizer.fit_on_texts(docs)

    # Integer Encode Tokens
    def integer_encode_documents(docs, tokenizer):
        return tokenizer.texts_to_sequences(docs)

    # Get Max Length Per Token
    def get_max_token_length_per_doc(docs: List[List[str]])-> int:
        return max(list(map(lambda x: len(x.split()), docs)))
    ## get the max length in terms of token length
    max_length = get_max_token_length_per_doc(docs)

    MAX_SEQUENCE_LENGTH = 300
    ## integer encode the documents
    encoded_docs = integer_encode_documents(docs, tokenizer)
    ## this is a list of lists, the numbers represent the index position of that word.
    ## for instance, 33 means the 33rd word in the vocabulary
    padded_docs = pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    
    return tokenizer, padded_docs, MAX_SEQUENCE_LENGTH

## Modeling

In [53]:
# before assigining 'other' category for brand
df['brand'].value_counts(normalize=True)[:50]

7 For All Mankind       0.146867
Rails                   0.046679
Intentionally Blank     0.041301
A.L.C.                  0.034097
Rachel Comey            0.033917
                          ...   
Two                     0.004629
Eleven Six              0.004564
Citizens of Humanity    0.004531
EMME PARSONS            0.004352
Banjanan                0.004238
Name: brand, Length: 50, dtype: float64

In [54]:
top50 = df['brand'].value_counts(normalize=True)[:50].index
def intop50(df):
    updatebrand = []
    for brand in df:
        if brand not in top50:
            updatebrand.append('other')
        else:
            updatebrand.append(brand)
    return updatebrand

Before we start training, let's take a look at the relative distribution of counts for cities. Baseline performance accuracy for this model should be the largest value **~14.7%**. You can get this accuracy by predicting the model.

In [55]:
df['brand'] = intop50(df['brand'])
df['brand'].value_counts(normalize=True)

7 For All Mankind       0.146867
other                   0.142515
Rails                   0.046679
Intentionally Blank     0.041301
A.L.C.                  0.034097
                          ...   
Two                     0.004629
Eleven Six              0.004564
Citizens of Humanity    0.004531
EMME PARSONS            0.004352
Banjanan                0.004238
Name: brand, Length: 51, dtype: float64

In [56]:
labels = df['brand']
encoder = LabelEncoder()
labels_encoded = to_categorical(encoder.fit_transform(labels))

### Final Model: 96.308%

Based on previous attempts, DNN model performs the best after word-embedding. Therefore, we decided to use it as our final model.

In [57]:
# define the model
def DNN(VOCAB_SIZE,EMBEDDING_SIZE,MAX_SEQUENCE_LENGTH):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE, input_length=MAX_SEQUENCE_LENGTH))
    model.add(Flatten()) 
    model.add(Dense(51, activation='sigmoid')) 
    return model

In [59]:
%%time
X = df['description']+' '+df['name']+' '+df['details']+' '+ df['attribute_pair']+' '+df['brand_category']

tokenizer, padded_docs, MAX_SEQUENCE_LENGTH = word_embedding_2(X)
VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)
EMBEDDING_SIZE = 100

final_dnn = DNN(VOCAB_SIZE,EMBEDDING_SIZE,MAX_SEQUENCE_LENGTH)
final_dnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels_encoded, test_size=0.2,stratify=labels_encoded,random_state=2021)
# fit the model
final_dnn.fit(X_train, y_train, validation_split = 0.1, epochs=20, verbose=1)

Epoch 1/20
1381/1381 [==============================] - 32s 23ms/step - loss: 0.0577 - acc: 0.5752 - val_loss: 0.0114 - val_acc: 0.9334
Epoch 2/20
1381/1381 [==============================] - 31s 23ms/step - loss: 0.0072 - acc: 0.9600 - val_loss: 0.0075 - val_acc: 0.9544
Epoch 3/20
1381/1381 [==============================] - 31s 23ms/step - loss: 0.0031 - acc: 0.9824 - val_loss: 0.0069 - val_acc: 0.9586
Epoch 4/20
1381/1381 [==============================] - 32s 23ms/step - loss: 0.0018 - acc: 0.9897 - val_loss: 0.0064 - val_acc: 0.9623
Epoch 5/20
1381/1381 [==============================] - 32s 23ms/step - loss: 0.0012 - acc: 0.9921 - val_loss: 0.0069 - val_acc: 0.9619
Epoch 6/20
1381/1381 [==============================] - 32s 23ms/step - loss: 8.7449e-04 - acc: 0.9945 - val_loss: 0.0076 - val_acc: 0.9617
Epoch 7/20
1381/1381 [==============================] - 32s 23ms/step - loss: 6.4238e-04 - acc: 0.9959 - val_loss: 0.0082 - val_acc: 0.9621
Epoch 8/20
1381/1381 [==================

In [60]:
# evaluate the model
loss, accuracy = final_dnn.evaluate(X_test, y_test, verbose=1) # it's on training data

print('-----------------------')
print('Accuracy: %f' % (accuracy*100))

384/384 [==============================] - 1s 2ms/step - loss: 0.0112 - acc: 0.9631
-----------------------
Accuracy: 96.308368


In [63]:
prediction = final_dnn.predict_classes(X_test)

In [65]:
predict = encoder.inverse_transform(prediction)
predict

array(['Chufy', 'other', '7 For All Mankind', ..., 'Cynthia Rowley',
       'other', 'Outerknown'], dtype=object)

In [67]:
actual = y_test.argmax(1)
actual_brand = encoder.inverse_transform(actual)

In [69]:
results = pd.DataFrame({'actual':actual_brand,'predict':predict})
results

,actual,predict
0,Chufy,Chufy
1,other,other
2,7 For All Mankind,7 For All Mankind
3,7 For All Mankind,7 For All Mankind
4,7 For All Mankind,7 For All Mankind
...,...,...
12266,7 For All Mankind,7 For All Mankind
12267,Misa,Misa
12268,Cynthia Rowley,Cynthia Rowley
12269,other,other


In [70]:
results['predict_true']=(results['actual']==results['predict'])

In [71]:
results.groupby(['actual'])['predict_true'].mean()

actual
6397                   0.953846
7 For All Mankind      0.991676
A.L.C.                 0.930788
ANINE BING             0.838235
ASTR the Label         0.961340
                         ...   
Varley US              0.988764
Want Les Essentiels    0.887324
Whit                   0.918367
lemlem                 0.958791
other                  0.971984
Name: predict_true, Length: 51, dtype: float64